<a href="https://colab.research.google.com/github/jacobeturpin/CRX14/blob/master/CRX14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>